In [1]:
# official modules
import argparse
import json
import os
import time
import numpy as np
from math import floor
from shutil import copy2
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import torch.distributed as dist
# self-defined module
from utils.helper import init_DDP, Logger, print_log, load_labels, build_model
from utils.data import dataloader
from utils.train_model import train_model
from utils.test_model import test_model


# packages for train_model.py
# official modules
import os
import time
from time import localtime, strftime
from math import ceil
import pickle5 as pickle
import numpy as np
from collections import defaultdict
import torch
from torch.cuda.amp import autocast, GradScaler
import torch.distributed as dist
# self-defined modules
from utils.loss import calculate_loss
from utils.helper import print_log, print_metrics, print_time, print_metric_format

/home/lingjia/anaconda3/envs/deepstorm3d/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description='3d localization')
# phase
parser.add_argument('--train_or_test', type=str, default='other', help='train or test')
parser.add_argument('--resume', action='store_true', default=False)
parser.add_argument('--gpu_number', type=str, default=None, help='assign gpu')
# data info
parser.add_argument('--num_im', type=int, default=None, help='Number of samples used, train:val=9:1')
parser.add_argument('--H', type=int, default=96, help='Height of image')
parser.add_argument('--W', type=int, default=96, help='Width of image')
parser.add_argument('--zmin', type=int, default=-20, help='min zeta')
parser.add_argument('--zmax', type=int, default=20, help='max zeta')
parser.add_argument('--clear_dist', type=int, default=1, help='safe margin for z axis')
parser.add_argument('--D', type=int, default=250, help='num grid of zeta axis')
parser.add_argument('--scaling_factor', type=int, default=800, help='entry value for existence of pts')
parser.add_argument('--upsampling_factor', type=int, default=2, help='grid dim=[H,W]*upsampling_factor')
# train info
parser.add_argument('--model_use', type=str, default='LocNet')
parser.add_argument('--postpro',  action='store_true', default=False, help='whether do post processing in dnn')
parser.add_argument('--batch_size', type=int, default=1, help='when training on multi GPU, is the batch size on each GPU')
parser.add_argument('--initial_learning_rate', type=float, default=None, help='initial learning rate for adam')
parser.add_argument('--lr_decay_per_epoch', type=int, default=None, help='number of epochs learning rate decay')
parser.add_argument('--lr_decay_factor', type=float, default=None, help='lr decay factor')
parser.add_argument('--max_epoch', type=int,   default=None, help='number of training epoches')
parser.add_argument('--save_epoch', type=int, default=None, help='save model per save_epoch')
# test info
parser.add_argument('--test_id_loc', type=str, default=None)
# path
parser.add_argument('--checkpoint_path', type=str,  default=None,  help='checkpoint to resume from')
parser.add_argument('--data_path', type=str, default='/home/lingjia/Documents/microscope/Data/training_images_zrange20', help='path for train and val data')
parser.add_argument('--save_path', type=str, default=None, help='path for save models and results')
# output
parser.add_argument('--name_time', type=str, default=None, help='string of running time')
# for nonconvex loss
parser.add_argument('--port', type=str, default=None, help='DDP master port')
parser.add_argument('--weight', type=str, default=None, help='lambda CEL0')
parser.add_argument('--extra_loss', type=str, default=None, help='indicate whether use cel0 for gaussian or nc for possion')
# for extra losses
parser.add_argument('--cel0_mu', type=float, default=None, help='mu in cel0 loss')
parser.add_argument('--klnc_a', type=float, default=None, help='a for nonconvex loss in KLNC')

# opt = parser.parse_args()
opt,_=parser.parse_known_args()

In [3]:
opt.H=96
opt.W=96
opt.zmin=-20
opt.zmax=20
opt.clear_dist=1
opt.D=250
opt.scaling_factor=800
opt.upsampling_factor=2
opt.model_use='LocNet'

opt.initial_learning_rate=1e-3
opt.lr_decay_per_epoch=7
opt.lr_decay_factor=0.5
opt.max_epoch=2
opt.save_epoch=10
opt.train_or_test='train'
opt.port='123789'

opt.ame_time="2099-12-12-12-12-12"
opt.num_im=10000
opt.data_path='/media/hdd/lingjia/hdd_rpsf/nonconvex_loss/data/gaussian_10k_pt50L5'
opt.save_path='/media/hdd/lingjia/hdd_rpsf/nonconvex_loss/temp'

opt.weight='1_1'
opt.extra_loss='mse3d_cel0'
opt.cel0_mu=0.001

opt.gpu_number = '0'

opt.batch_size=24

In [4]:
rank = 0
world_size = 1

opt.rank = rank
opt.world_size = world_size

init_DDP(opt)


In [5]:
# calculate zoom ratio of z-axis
opt.pixel_size_axial = (opt.zmax - opt.zmin + 1 + 2*opt.clear_dist) / opt.D

# split dataset to train, validation 9:1
train_IDs = np.arange(1,floor(opt.num_im*0.9)+1,1).tolist()
val_IDs = np.arange(floor(opt.num_im*0.9)+1,opt.num_im+1).tolist()

opt.partition = {'train': train_IDs, 'valid': val_IDs}
opt.ntrain, opt.nval = len(train_IDs), len(val_IDs)

# output folder name
name_time = opt.name_time if opt.name_time else time.strftime('%Y-%m-%d-%H-%M-%S')
save_name = name_time + '-lr'+str(opt.initial_learning_rate) + \
    '-bs'+str(opt.batch_size) + \
    '-D'+str(opt.D) + \
    '-Ep'+str(opt.max_epoch) + \
    '-nT'+str(opt.ntrain)
if opt.extra_loss:
    save_name = save_name + '-w' + str(opt.weight) + '-' + str(opt.extra_loss)
if opt.cel0_mu:
    save_name = save_name + '-' + str(opt.cel0_mu)
if opt.klnc_a:
    save_name = save_name + '-' + str(opt.klnc_a)
save_name = save_name + '-' + str(opt.model_use)

if opt.resume:
    save_name = save_name + '-resume'
if opt.postpro:
    save_name = save_name + '-postpro'
opt.save_path = os.path.join(opt.save_path,save_name)
os.makedirs(opt.save_path, exist_ok=True)

# log files
if rank == 0:
    log = open(os.path.join(opt.save_path, '{}_log.txt'.format(time.strftime('%H-%M-%S'))), 'w')
    logger = Logger(os.path.join(opt.save_path, '{}_tensorboard'.format(time.strftime('%H-%M-%S'))))

    print_log('[INFO==>] setup_params:',log)
    # for key,value in opt._get_kwargs():
        # if not key == 'partition':
            # print_log('{}: {}'.format(key,value),log)
    print_log(f'[INFO==>] Dataset: Train {len(train_IDs)} Val {len(val_IDs)}',log)


device = torch.device('cuda')
# what is this?
torch.backends.cudnn.benchmark = True


if opt.rank==0:
    # save setup parameters in result folder as well
    with open(os.path.join(opt.save_path,'setup_params.json'),'w') as handle:
        json.dump(opt.__dict__, handle, indent=2)

# Load labels and generate dataset
labels = load_labels(os.path.join(opt.data_path,'label.txt'))

# Parameters for dataloaders
params_train = {'batch_size': opt.batch_size, 'shuffle': True,  'partition': opt.partition['train']}
params_val = {'batch_size': opt.batch_size, 'shuffle': False, 'partition': opt.partition['valid']}

training_generator = dataloader(opt.data_path, labels, params_train, opt, num_workers=0)
validation_generator = dataloader(opt.data_path, labels, params_val, opt, num_workers=0)

# model
model = build_model(opt)
model.to(device)
model = DDP(model,find_unused_parameters=True,broadcast_buffers=False)

optimizer = Adam(list(model.parameters()), lr=opt.initial_learning_rate)

# opt.scheduler_type = 'ReduceLROnPlateau'
# if opt.scheduler_type == 'StepLR':
# scheduler = lr_scheduler.StepLR(optimizer, step_size=opt.lr_decay_per_epoch, gamma=opt.lr_decay_factor)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=opt.lr_decay_factor, patience=opt.lr_decay_per_epoch)

# Print Model layers and number of parameters
if opt.rank == 0:
    # print_log(model, log)
    print_log("[INFO==>] Number of parameters: {}".format(sum(param.numel() for param in model.parameters())),log)


# if resume_training, continue from a checkpoint
if opt.resume:
    print_log("[INFO==>] Resume...",log)
    checkpoint = torch.load(opt.checkpoint_path)
    model.load_state_dict(checkpoint['model'])
    # optimizer.load_state_dict(checkpoint['optimizer'])

[INFO==>] setup_params:
[INFO==>] Dataset: Train 9000 Val 1000
[INFO==>] Number of parameters: 1594162


In [6]:
learning_results = defaultdict(list)
max_epoch = opt.max_epoch

steps_train = ceil(opt.ntrain / opt.batch_size / opt.world_size)
steps_val = ceil(opt.nval / opt.batch_size / opt.world_size)
params_val = {'batch_size': opt.batch_size, 'shuffle': False}

# loss function
loss_type = ['loss']
if opt.extra_loss: # None or string
    extra_loss = opt.extra_loss.split('_')
    extra_weight = [float(n) for n in opt.weight.split('_')]
    loss_type = loss_type + extra_loss
    if not len(extra_loss) == len(extra_weight):
        raise Exception(f'Input {len(extra_loss)} weight with {len(extra_weight)} extra loss')
print_log(f'[INFO==>] Loss types: {loss_type}',log)
calc_loss = calculate_loss(opt,loss_type,extra_weight)

scaler = GradScaler()

# start from scratch
start_epoch, end_epoch = 0, max_epoch
learning_results = {'val_max': [], 'val_sum': [], 'steps_per_epoch': steps_train}
for loss in loss_type:
    learning_results['train_'+loss] = []
    learning_results['val_'+loss] = []
best_val_loss = float('Inf')


# starting time of training
train_start_time = time.time()
not_improve = 0

print_log(f'[INFO==>] Start training from {start_epoch} to {end_epoch} rank {opt.rank}\n',log)

epoch = 0

if opt.rank == 0:
    print_log(f'Epoch {epoch+1}/{end_epoch} | {strftime("%Y-%m-%d %H:%M:%S", localtime())} | lr {optimizer.param_groups[0]["lr"]}', log, arrow=True)

# training phase
model.train()
metric, metrics = defaultdict(float), defaultdict(float)

[INFO==>] Loss types: ['loss', 'mse3d', 'cel0']
[INFO==>] Start training from 0 to 2 rank 0

---> Epoch 1/2 | 2022-10-16 20:30:02 | lr 0.001


In [7]:
# starting time of current epoch
epoch_start_time = time.time()

if opt.rank == 0:
    print_log(f'Epoch {epoch+1}/{end_epoch} | {strftime("%Y-%m-%d %H:%M:%S", localtime())} | lr {optimizer.param_groups[0]["lr"]}', log, arrow=True)

# training phase
model.train()
metric, metrics = defaultdict(float), defaultdict(float)
with torch.set_grad_enabled(True):
    for batch_ind, (inputs, targets, target_ims, fileids) in enumerate(training_generator):

        inputs = inputs.to(device)
        targets = targets.to(device)
        target_ims = target_ims.to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
        loss = calc_loss(upgrid=outputs.float(), gt_upgrid=targets, gt_im=target_ims, metric=metric, metrics=metrics)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if opt.rank==0:
            print(f'Epoch {epoch}/{end_epoch-1} Train {batch_ind}/{steps_train-1} MaxOut {outputs.max():.2f} {print_metric_format(metric)}')


# if opt.scheduler_type == 'StepLR':
    # scheduler.step()

if opt.gpu_number:
    for key in metrics.keys():
        dist.all_reduce_multigpu([metrics[key]])

# calculate and print all keys in metrics
if opt.rank == 0:
    print_metrics(metrics, opt.ntrain, 'Train', log)

# record training loss
for key in loss_type:
    mean_train_tmp = (metrics[key]/opt.ntrain).cpu().numpy()
    learning_results['train_'+key].append(mean_train_tmp)




""" validation """
model.eval()
metrics_val = defaultdict(float)

with torch.set_grad_enabled(False):
    for batch_ind, (inputs, targets, target_ims, fileids) in enumerate(validation_generator):

        inputs = inputs.to(device)
        targets = targets.to(device)
        target_ims = target_ims.to(device)

        # forward
        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs)
        val_loss = calc_loss(upgrid=outputs.float(), gt_upgrid=targets, gt_im=target_ims, metric=metric, metrics=metrics_val)

        if opt.rank==0:
            print(f'Epoch {epoch+1}/{end_epoch} Val {batch_ind}/{steps_val-1} MaxOut {outputs.max():.2f} {print_metric_format(metric)}')

if opt.gpu_number:
    for key in metrics_val.keys():
        dist.all_reduce_multigpu([metrics_val[key]])

# calculate and print all keys in metrics_val
if opt.rank == 0:
    print_metrics(metrics_val, opt.nval, 'Valid',log)

# record validation loss
for key in loss_type:
    mean_train_tmp = (metrics_val[key]/opt.ntrain).cpu().numpy()
    learning_results['val_'+key].append(mean_train_tmp)

# if not opt.scheduler_type == 'StepLR':
mean_val_loss = (metrics_val['loss']/opt.nval).cpu().numpy()
scheduler.step(mean_val_loss)

# sanity check: record maximal value and sum of last validation sample
max_last = outputs.max().cpu().numpy()
sum_last = (outputs.sum()/params_val['batch_size']).cpu().numpy()
learning_results['val_max'].append(max_last)
learning_results['val_sum'].append(sum_last)



# visualize in tensorboard
if opt.rank==0:
    for key in metrics.keys():
        logger.scalars_summary('LOSS/{}'.format(key) ,{'train':metrics[key]/opt.ntrain, 'val':metrics_val[key]/opt.nval}, epoch)
    logger.scalar_summary('Other/MaxOut', max_last/opt.scaling_factor, epoch)
    logger.scalar_summary('Other/MaxOutSum', sum_last, epoch)
    logger.scalar_summary('Other/Learning Rate', optimizer.param_groups[0]['lr'], epoch)


# save checkpoint
if opt.rank==0:
    # save latest checkpoint
    torch.save({
        'epoch': epoch + 1,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict()}, os.path.join(opt.save_path,'ckpt_latest'))
    # save checkpoint per save_epoch
    if epoch%opt.save_epoch == opt.save_epoch-1:
        torch.save({
            'epoch': epoch + 1,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()}, os.path.join(opt.save_path,'ckpt_'+str(epoch+1)))

# save checkpoint for best val loss
if mean_val_loss < (best_val_loss - 1e-4):
    if opt.rank==0:
        # print an update and save the model weights
        print_log('Val loss improved from %.4f to %.4f, saving best model...'% (best_val_loss, mean_val_loss), log, arrow=True)
        torch.save({
            'epoch': epoch + 1,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict()}, os.path.join(opt.save_path,'ckpt_best_loss'))
    # change minimal loss and init stagnation indicator
    best_val_loss = mean_val_loss
    not_improve = 0
else:
    # update stagnation indicator
    not_improve += 1
    if opt.rank==0:
        print_log('Val loss not improve by %d epochs, best val loss: %.4f'% (not_improve,best_val_loss), log, arrow=True)

epoch_time_elapsed = time.time() - epoch_start_time
if opt.rank==0:
    print_log('Max test last: %.2f, Sum test last: %.2f' %(max_last, sum_last), log, arrow=True)
    print_log('{}, Epoch complete in {}\n'.format(time.ctime(time.time()),print_time(epoch_time_elapsed)), log, arrow=True)

    # save all records for latter visualization
    with open(os.path.join(opt.save_path, 'learning_results.pickle'), 'wb') as handle:
        pickle.dump(learning_results, handle, protocol=pickle.HIGHEST_PROTOCOL)


---> Epoch 1/2 | 2022-10-16 20:30:03 | lr 0.001
Epoch 0/1 Train 0/374 MaxOut 38.41 mse3d 55.6015  cel0 0.0010  loss 55.6025  


RuntimeError: CUDA out of memory. Tried to allocate 810.00 MiB (GPU 0; 10.76 GiB total capacity; 8.22 GiB already allocated; 432.44 MiB free; 9.46 GiB reserved in total by PyTorch)

: 

In [8]:
if opt.gpu_number:
    for key in metrics.keys():
        dist.all_reduce_multigpu([metrics[key]])

# calculate and print all keys in metrics
if opt.rank == 0:
    print_metrics(metrics, opt.ntrain, 'Train', log)

# record training loss
for key in loss_type:
    mean_train_tmp = (metrics[key]/opt.ntrain).cpu().numpy()
    learning_results['train_'+key].append(mean_train_tmp)

---> Train mse3d: 0.0003, cel0: 0.0000, loss: 0.8709


In [8]:
""" Functions """
# official modules
import numpy as np
import os
import torch
from torch.utils.tensorboard import SummaryWriter
import torch.distributed as dist
# self-defined modules
from utils.networks import *

def init_DDP(opt):
    os.environ['MASTER_ADDR']='localhost'
    os.environ['MASTER_PORT']='123458'
    gpus = [g.strip() for g in opt.gpu_number.split(',')]
    os.environ['CUDA_VISIBLE_DEVICES']=gpus[opt.rank]
    dist.init_process_group('GLOO',rank=opt.rank,world_size=opt.world_size)

In [9]:
gpu_number = len(opt.gpu_number.split(','))
# print(gpu_number)
opt.rank = 0
opt.world_size = 1
init_DDP(opt)

In [10]:
tmp_train = np.arange(1,9001,1).tolist()
tmp_val = np.arange(9001,10001,1).tolist()
train_IDs = [str(i) for i in tmp_train]
val_IDs = [str(i) for i in tmp_val]

opt.partition = {'train': train_IDs, 'valid': val_IDs}
opt.ntrain, opt.nval = len(train_IDs), len(val_IDs)

# calculate zoom ratio of z-axis
opt.pixel_size_axial = (opt.zmax - opt.zmin + 1 + 2*opt.clear_dist) / opt.D

# output folder name for results
t = time.strftime('%m%d') + \
    '-lr'+str(opt.initial_learning_rate) + \
    '-batchSize'+str(opt.batch_size) + \
    '-D'+str(opt.D) + \
    '-Epoch'+str(opt.max_epoch) + \
    '-nTrain'+str(opt.ntrain) + \
    '-'+str(opt.model_use)

if opt.resume:
    t = t + '-resume'
if opt.postpro:
    t = t + '-postpro'

opt.save_path = os.path.join(opt.save_path,t)
print(opt.save_path)
# os.makedirs(opt.save_path, exist_ok=True)

/media/hdd/lingjia/rPSF/result/CEL0/0613-lr0.0005-batchSize1-D250-Epoch30-nTrain9000-cnn_residual


In [ ]:
device = torch.device('cuda')
# what is this?
torch.backends.cudnn.benchmark = True

if opt.rank==0:
    # save setup parameters in result folder as well
    with open(os.path.join(opt.save_path,'setup_params.json'),'w') as handle:
        json.dump(opt.__dict__, handle, indent=2)

# Load labels and generate dataset
labels = load_labels(os.path.join(opt.data_path,'label.txt'))

# Parameters for dataloaders
params_train = {'batch_size': opt.batch_size, 'shuffle': True,  'partition': opt.partition['train']}
params_val = {'batch_size': opt.batch_size, 'shuffle': False, 'partition': opt.partition['valid']}

training_generator = dataloader(opt.data_path, labels, params_train, opt, num_workers=0)
validation_generator = dataloader(opt.data_path, labels, params_val, opt, num_workers=0)


In [11]:

spikes_pred = torch.zeros((96,96,41))
spikes_pred[0,0,0] = 1

norm_ai = torch.ones((96,96,41))

# CEL0 on the predicted spikes
norm_ai2 = tf.square(norm_ai)
thresh = np.sqrt(2*weight)/norm_ai
abs_heat = tf.abs(spikes_pred)
bound = tf.square(abs_heat-thresh)
ind = tf.cast(abs_heat <= thresh, tf.float32)
loss_spikes = tf.reduce_mean((weight-0.5*(norm_ai2*bound)*ind))




In [15]:
class CEL0Loss(nn.Module):
    def __init__(self,norm_ai,weight):
        super(CEL0Loss,self).__init__()
        norm_ai = norm_ai
        weight = weight

    def forward(self,spikes_pred):
        # CEL0 on the predicted spikes
        norm_ai2 = torch.square(norm_ai)
        thresh = np.sqrt(2*weight)/norm_ai
        abs_heat = torch.abs(spikes_pred)
        bound = torch.square(abs_heat-thresh)
        ind =(abs_heat<= thresh).type(torch.float32)
        loss_spikes = torch.mean((weight-0.5*(norm_ai2*bound)*ind))

        return loss_spikes

In [16]:
cel0Loss = CEL0Loss(norm_ai=torch.ones(96,96,41),weight=100)

In [21]:
spikes_pred = torch.zeros((96,96,41))
spikes_pred[:30,:,:20] = 1
loss = cel0Loss(spikes_pred)
print(loss)

tensor(2.0796)


In [24]:
import scipy
norm_ai_path = '/home/lingjia/Documents/rPSF/NN/utils/norm_ai.mat'
norm_ai =  scipy.io.loadmat(norm_ai_path)['a']
norm_ai =  torch.from_numpy(norm_ai)
# print(type(norm_ai))
print(norm_ai.shape)


torch.Size([96, 96, 41])


In [31]:
norm_ai = torch.randn((2,2))
x = torch.randn((3,2,2))
weight = 100
print(norm_ai,'\n',x)

tensor([[-0.6309,  0.1307],
        [ 1.9685, -1.3234]]) 
 tensor([[[-0.3528, -1.1910],
         [-0.7360, -0.5470]],

        [[ 1.3621, -0.4500],
         [ 2.7612, -0.5511]],

        [[ 0.8429,  0.6193],
         [-0.3980,  2.0948]]])


In [32]:
norm_ai2 = torch.square(norm_ai)
print(norm_ai2.shape)
print(norm_ai2)


torch.Size([2, 2])
tensor([[0.3981, 0.0171],
        [3.8751, 1.7513]])


In [33]:
thresh = np.sqrt(2*weight)/norm_ai
print(thresh)

tensor([[-22.4151, 108.2227],
        [  7.1841, -10.6864]])


In [35]:
abs_heat = torch.abs(x)
print(abs_heat)

tensor([[[0.3528, 1.1910],
         [0.7360, 0.5470]],

        [[1.3621, 0.4500],
         [2.7612, 0.5511]],

        [[0.8429, 0.6193],
         [0.3980, 2.0948]]])


In [36]:
bound = torch.square(abs_heat-thresh)
print(bound)


tensor([[[  518.3761, 11455.7861],
         [   41.5781,   126.1883]],

        [[  565.3557, 11614.9531],
         [   19.5626,   126.2795]],

        [[  540.9315, 11578.5098],
         [   46.0515,   163.3588]]])


In [37]:
ind = (abs_heat<= thresh).type(torch.float32)
print(ind)


tensor([[[0., 1.],
         [1., 0.]],

        [[0., 1.],
         [1., 0.]],

        [[0., 1.],
         [1., 0.]]])


In [41]:
print(norm_ai2)
print(bound)
print(0.5*(norm_ai2*bound)*ind)
print((weight-0.5*(norm_ai2*bound)*ind))

tensor([[0.3981, 0.0171],
        [3.8751, 1.7513]])
tensor([[[  518.3761, 11455.7861],
         [   41.5781,   126.1883]],

        [[  565.3557, 11614.9531],
         [   19.5626,   126.2795]],

        [[  540.9315, 11578.5098],
         [   46.0515,   163.3588]]])
tensor([[[ 0.0000, 97.8110],
         [80.5596,  0.0000]],

        [[ 0.0000, 99.1700],
         [37.9035,  0.0000]],

        [[ 0.0000, 98.8589],
         [89.2271,  0.0000]]])
tensor([[[100.0000,   2.1890],
         [ 19.4404, 100.0000]],

        [[100.0000,   0.8300],
         [ 62.0965, 100.0000]],

        [[100.0000,   1.1411],
         [ 10.7729, 100.0000]]])


In [43]:
loss_spikes = torch.mean((weight-0.5*(norm_ai2*bound)*ind))
print(loss_spikes)

tensor(58.0392)


In [5]:
import scipy.io
import torch
import numpy as np
# norm_ai_path = './matlab_codes/norm_ai_test.mat'dl
# norm_ai = scipy.io.loadmat(norm_ai_path)['norm_ai_test']
weight = 0.1
# spikes_pred = torch.randn((96,96,21))

norm_ai = torch.randn((2,2,3))
spikes_pred = torch.randn((2,2,3))

norm_ai2 = torch.square(norm_ai)
thresh = np.sqrt(2*weight)/norm_ai
abs_heat = torch.abs(spikes_pred)
bound = torch.square(abs_heat-thresh)
ind = (abs_heat<=thresh).type(torch.float32)
loss_spikes = torch.mean((weight-0.5*(norm_ai2*bound)*ind))



In [6]:
print(f'norm_ai:\n{norm_ai}')
print(f'norm_ai2:\n{norm_ai2}')

norm_ai:
tensor([[[-0.5372, -0.7486, -0.6857],
         [ 0.1539, -1.6697,  0.4003]],

        [[-1.3305, -0.4892,  0.3958],
         [ 0.0053, -0.0951,  0.9309]]])
norm_ai2:
tensor([[[2.8856e-01, 5.6036e-01, 4.7023e-01],
         [2.3683e-02, 2.7878e+00, 1.6024e-01]],

        [[1.7704e+00, 2.3927e-01, 1.5665e-01],
         [2.8479e-05, 9.0473e-03, 8.6658e-01]]])


In [2]:
loss_type = ['loss','mse3d','cel0']
losses = ['train_'+n for n in loss_type]+['val_'+n for n in loss_type]
# learning_results = {'train_loss': [], 'val_loss': [],
#                     'train_mse3d': [], 'train_mse2d': [],
#                     'train_dice': [], 'val_dice': [], 
#                     'train_mse2d': [], 'val_mse3d': [], 
#                     'train_CEL0': [], 'val_CEL0': [], 
#                     'val_max': [], 'val_sum': [], 'steps_per_epoch': steps_train}

steps_train = 1
learning_results = {'val_max': [], 'val_sum': [], 'steps_per_epoch': steps_train}

for loss in losses:
    learning_results[loss] = []

        

In [19]:
def print_metric(metric):
    l = len(metric)
    info = ''
    for key in metric.keys():
        info = info + f'{key}: {metric[key]:.4f}  '
    print(info)

metric = {'a':0.1,'bb':0.097}
print_metric(metric)

a: 0.1000  bb: 0.0970  


In [10]:
import torch
import numpy
a = torch.Tensor([[0,1,2],[3,4,5],[6,7,8]])
a = numpy.array([[0,1,2],[3,4,5],[6,7,8]])
print(a)
print(a.shape)
b = numpy.tile(a,(1,1,2))
print(b)
print(b.shape)


[[0 1 2]
 [3 4 5]
 [6 7 8]]
(3, 3)
[[[0 1 2 0 1 2]
  [3 4 5 3 4 5]
  [6 7 8 6 7 8]]]
(1, 3, 6)


In [18]:
a = torch.Tensor([[0,1,2],[3,4,5],[6,7,8]])
a = a.unsqueeze_(0)
print(a.shape)
b = a.expand(2,3,3)
print(b.shape)
print(b)
# print(b[:,:,0])
# print(b.permute(2,0,1))

torch.Size([1, 3, 3])
torch.Size([2, 3, 3])
tensor([[[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]],

        [[0., 1., 2.],
         [3., 4., 5.],
         [6., 7., 8.]]])


In [2]:
import torch
a = 3
input = torch.randn(2,3,3)
print(f'input:\n{input}')
output = input/(a+input)
print(f'output:\n{output}')
out = torch.sum(output)
print(out)

input:
tensor([[[-0.1044, -1.3292, -0.6696],
         [ 2.2702,  3.1764, -1.0143],
         [-0.1016,  0.7238,  1.2969]],

        [[-0.0660,  0.2672,  0.3975],
         [-0.3368,  1.4430,  0.6608],
         [ 1.2340,  1.1386,  1.3968]]])
output:
tensor([[[-0.0360, -0.7955, -0.2873],
         [ 0.4308,  0.5143, -0.5108],
         [-0.0350,  0.1944,  0.3018]],

        [[-0.0225,  0.0818,  0.1170],
         [-0.1265,  0.3248,  0.1805],
         [ 0.2914,  0.2751,  0.3177]]])
tensor(1.2158)


In [11]:
a = '/media/hdd/lingjia/hdd_rpsf/nonconvex_loss/temp/2022-09-24-17-44-34-lr0.001-bs16-D41-Ep5-nT90-w1.0-deepstorm3d'
print(a.split('/')[-1][:])
weight = a.split('w')[-1].split('-')[0]
nc = 'klnc'
weight+nc

a = '/media/hdd/lingjia/hdd_rpsf/data/plain/test/test_plain/test10'
a.split('/')[-1][4:]

2022-09-24-17-44-34


'10'

In [4]:
from time import localtime, strftime
strftime("%Y-%m-%d %H:%M:%S", localtime())

'2022-10-08 13:58:02'

In [2]:
loss_type = ['loss','mse3d']
extra_loss = 'loss1_loss2_loss3'
if extra_loss: # None or string
    extra_loss = extra_loss.split('_')
    loss_type = loss_type + extra_loss
print(loss_type)

['loss', 'mse3d', 'loss1', 'loss2', 'loss3']


In [17]:
import scipy.io
D = 127
# norm_ai_path = f'/home/lingjia/Documents/rpsf/3DLocalization/matlab_codes/norm_ai_D41.mat'
# norm_ai_path = '/home/lingjia/Documents/rpsf/3DLocalization/matlab_codes/norm_ai.mat'
norm_ai_path = f'./matlab_codes/norm_ai_D{D}.mat'
print(norm_ai_path)
norm_ai = scipy.io.loadmat(norm_ai_path)['norm_ai']
print(norm_ai.shape)

./matlab_codes/norm_ai_D127.mat
(96, 96, 127)
